In [1]:
import numpy as np
from scipy.special import psi
from math import log, exp

def discrete_continuous_info(d, c, k=3, base=exp(1)):
    """
    Estimates mutual information between a discrete vector 'd' and continuous vectors 'c'.
    
    Args:
        d (np.ndarray): Discrete array of shape (1, N) or (N,)
        c (np.ndarray): Continuous data of shape (M, N) where M is feature dim
        k (int): Number of nearest neighbors
        base (float): Logarithm base (default: natural log)
    
    Returns:
        f (float): Estimated mutual information
        V (np.ndarray): Volume estimates for each sample
    """
    d = np.asarray(d)
    c = np.asarray(c)

    if d.ndim == 1:
        d = d.reshape(1, -1)
    if c.ndim == 1:
        c = c.reshape(1, -1)

    N = c.shape[1]
    symbol_IDs = np.zeros(N, dtype=int)
    first_symbol = []
    c_split = []
    cs_indices = []
    num_d_symbols = 0

    # Bin continuous data by discrete symbols
    for i in range(N):
        found = False
        for j in range(num_d_symbols):
            if d[:, i] == d[:, first_symbol[j]]:
                symbol_IDs[i] = j
                found = True
                break
        if not found:
            symbol_IDs[i] = num_d_symbols
            first_symbol.append(i)
            c_split.append([])
            cs_indices.append([])
            num_d_symbols += 1

        c_split[symbol_IDs[i]].append(c[:, i])
        cs_indices[symbol_IDs[i]].append(i)

    # Convert to numpy arrays
    for i in range(num_d_symbols):
        c_split[i] = np.column_stack(c_split[i])

    m_tot = 0
    av_psi_Nd = 0
    V = np.zeros(N)
    all_c_distances = np.zeros(N)
    psi_ks = 0

    for c_bin in range(num_d_symbols):
        n_bin = c_split[c_bin].shape[1]
        one_k = min(k, n_bin - 1)

        if one_k > 0:
            for pivot in range(n_bin):
                # Compute distances within bin
                c_pivot = c_split[c_bin][:, pivot]
                c_distances = np.linalg.norm(c_split[c_bin] - c_pivot[:, None], axis=0)
                sorted_distances = np.sort(c_distances)
                eps_over_2 = sorted_distances[one_k + 1]  # skip pivot

                # Count total points within volume
                all_distances = np.linalg.norm(c - c_pivot[:, None], axis=0)
                m = max(np.sum(all_distances <= eps_over_2) - 1, 0)

                m_tot += psi(m)
                V[cs_indices[c_bin][pivot]] = (2 * eps_over_2) ** d.shape[0]
        else:
            m_tot += psi(num_d_symbols * 2)

        p_d = n_bin / N
        av_psi_Nd += p_d * psi(p_d * N)
        psi_ks += p_d * psi(max(one_k, 1))

    f = (psi(N) - av_psi_Nd + psi_ks - m_tot / N) / log(base)
    return f, V


In [8]:
import numpy as np
from scipy.spatial import distance
from scipy.special import digamma
from math import log, exp

def discrete_continuous_info_2(d, c, k=3, base=np.e):
    """
    Estimates the mutual information between a discrete vector `d`
    and a continuous matrix `c` using a nearest-neighbor approach.
    
    Parameters:
    d (np.ndarray): 1D array of discrete labels (shape: [n_samples])
    c (np.ndarray): 2D array of continuous data (shape: [n_features, n_samples])
    k (int): number of nearest neighbors (default: 3)
    base (float): log base for the mutual information (default: natural log)
    
    Returns:
    f (float): estimated mutual information
    V (np.ndarray): estimated local volume per point
    """
    n_samples = c.shape[1]
    symbol_IDs = np.zeros(n_samples, dtype=int)
    c_split = {}
    cs_indices = {}
    unique_labels = []
    num_d_symbols = 0

    # Bin the data by discrete labels
    for i in range(n_samples):
        label = d[i]
        if label not in unique_labels:
            unique_labels.append(label)
            num_d_symbols += 1
            c_split[label] = []
            cs_indices[label] = []
        c_split[label].append(c[:, i])
        cs_indices[label].append(i)

    for label in c_split:
        c_split[label] = np.column_stack(c_split[label])  # Convert to numpy array

    m_tot = 0
    av_psi_Nd = 0
    V = np.zeros(n_samples)
    psi_ks = 0

    for label in c_split:
        group_c = c_split[label]
        group_indices = cs_indices[label]
        n_group = group_c.shape[1]
        one_k = min(k, n_group - 1)

        if one_k > 0:
            for pivot in range(n_group):
                pivot_vec = group_c[:, pivot].reshape(-1, 1)
                diffs = group_c - pivot_vec
                distances = np.linalg.norm(diffs, axis=0)
                sorted_distances = np.sort(distances)
                eps_over_2 = sorted_distances[one_k + 1]  # k-th neighbor (skip self)

                # Count neighbors in all data within this radius
                full_diffs = c - pivot_vec
                all_distances = np.linalg.norm(full_diffs, axis=0)
                m = max(np.sum(all_distances <= eps_over_2) - 1, 0)
                m_tot += digamma(m + 1)  # +1 for numerical stability
                V[group_indices[pivot]] = (2 * eps_over_2) ** c.shape[0]
        else:
            m_tot += digamma(num_d_symbols * 2)

        p_d = n_group / n_samples
        av_psi_Nd += p_d * digamma(n_group)
        psi_ks += p_d * digamma(max(one_k, 1))

    f = (digamma(n_samples) - av_psi_Nd + psi_ks - m_tot / n_samples) / log(base)
    return f, V


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# --- Step 1: Generate synthetic data ---
np.random.seed(42)
N = 300  # number of samples
dim = 2  # dimensionality of continuous variable

# Create 3 discrete classes
n_classes = 3
samples_per_class = N // n_classes

c = []
d = []

for i in range(n_classes):
    # Continuous values clustered around different means
    mean = np.random.randn(dim) * 5
    cov = np.eye(dim) * 0.5
    cluster = np.random.multivariate_normal(mean, cov, size=samples_per_class).T
    c.append(cluster)
    d += [i] * samples_per_class

# Stack all continuous samples into a (dim, N) array
c = np.hstack(c)
d = np.array(d).reshape(1, -1)

# --- Step 2: Estimate mutual information ---
mi, V = discrete_continuous_info(d, c, k=3, base=np.e)
print(f"Estimated Mutual Information I(D; C): {mi:.4f}")
mi, V = discrete_continuous_info_2(d.reshape(-1), c, k=3, base=np.e)
print(f"Estimated Mutual Information I(D; C): {mi:.4f}")

# --- Step 3: Optional - Visualize data ---
plt.figure(figsize=(6, 5))
colors = ['red', 'green', 'blue']
for i in range(n_classes):
    class_data = c[:, d.flatten() == i]
    plt.scatter(class_data[0], class_data[1], label=f'Class {i}', alpha=0.7, color=colors[i])
plt.title("Synthetic 2D Continuous Data by Class")
plt.xlabel("x1")
plt.ylabel("x2")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


Estimated Mutual Information I(D; C): 0.7686


TypeError: unhashable type: 'numpy.ndarray'

In [4]:
import numpy as np
from scipy.special import psi


def findpt(c_sorted, target):
    """
    Find the "index" of where `target` would go in sorted array `c_sorted`.
    This mimics MATLAB's 0.5 behavior between indices.
    """
    if target < c_sorted[0]:
        return 0.5
    elif target > c_sorted[-1]:
        return len(c_sorted) + 0.5
    else:
        return np.searchsorted(c_sorted, target, side='left')


def discrete_continuous_info_fast(d, c, k=3, base=np.e):
    d = np.asarray(d).flatten()
    c = np.asarray(c).flatten()
    
    assert len(d) == len(c), "d and c must be the same length"

    # Sort c and reorder d accordingly
    idx = np.argsort(c)
    c = c[idx]
    d = d[idx]

    # Bin the continuous data by discrete labels
    unique_labels, symbol_IDs = np.unique(d, return_inverse=True)
    num_d_symbols = len(unique_labels)

    c_split = [[] for _ in range(num_d_symbols)]
    cs_indices = [[] for _ in range(num_d_symbols)]
    
    for i in range(len(d)):
        label_id = symbol_IDs[i]
        c_split[label_id].append(c[i])
        cs_indices[label_id].append(i)

    c_split = [np.array(cs) for cs in c_split]
    V = np.zeros(len(d))
    m_tot = 0
    av_psi_Nd = 0
    psi_ks = 0

    for c_bin in range(num_d_symbols):
        bin_c = c_split[c_bin]
        indices = cs_indices[c_bin]
        bin_len = len(bin_c)
        one_k = min(k, bin_len - 1)

        if one_k > 0:
            for pivot in range(bin_len):
                one_c = bin_c[pivot]
                left = pivot
                right = pivot

                # Find the k-th nearest neighbor (1D)
                for _ in range(one_k):
                    if left == 0:
                        right += 1
                    elif right == bin_len - 1:
                        left -= 1
                    else:
                        if abs(bin_c[left - 1] - one_c) < abs(bin_c[right + 1] - one_c):
                            left -= 1
                        else:
                            right += 1

                # Radius of neighborhood
                distance_to_neighbor = abs(bin_c[right] - one_c) if right > pivot else abs(bin_c[left] - one_c)

                # Count number of total samples in full c within the same radius
                if right > pivot:
                    m = int(np.floor(findpt(c, bin_c[right]) - findpt(c, one_c - distance_to_neighbor)))
                else:
                    m = int(np.floor(findpt(c, one_c + distance_to_neighbor) - findpt(c, bin_c[left])))

                if m < one_k:
                    m = one_k

                m_tot += psi(m)
                V[indices[pivot]] = 2 * distance_to_neighbor
        else:
            m_tot += psi(num_d_symbols * 2)
            V[indices[0]] = 2 * (c[-1] - c[0])

        p_d = bin_len / len(d)
        av_psi_Nd += p_d * psi(p_d * len(d))
        psi_ks += p_d * psi(max(one_k, 1))

    f = (psi(len(d)) - av_psi_Nd + psi_ks - m_tot / len(d)) / np.log(base)
    return f, V


In [7]:
# Example with synthetic data
np.random.seed(1)
N = 300
d = np.random.choice([0, 1, 2], size=N)
c = np.random.randn(N) + d  # continuous variable depends on discrete label

mi, V = discrete_continuous_info_fast(d, c)
print(f"Estimated MI (fast version): {mi:.4f}")

Estimated MI (fast version): 0.4344
